In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt
import random
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from matplotlib import colors
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

# Ejercicio 1

Se está diseñando un web service, el cual cada vez que es invocado consulta a una base de datos.
Se estima que el tiempo que transcurre entre cada llamada al servicio se puede modelar según una distribución
exponencial con media μ = 4 segundos. Se debe decidir la arquitectura de base de datos a utilizar entre las dos siguientes:

1) Utilizar 2 bases de datos distribuidas.
Con probabilidad p = 0.6 las solicitudes son atendidas por la base 1 y con probabilidad q = 1 − p son atendidos por
la base de datos 2.
El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con
medias, μ 1 = 0,7 seg y μ 2 = 1 seg respectivamente.

2) Utilizar 1 base de datos central.
En este caso la demora en resolver una solicitud sigue una distribución exponencial con μ = 0,8 segundos.

Simular para cada opción 100000 solicitudes procesadas, determinando:

 - El tiempo medio de espera entre que la solicitud llega y puede ser procesada (suponer que ninguna conexión se
cae por timeout).

 - La fracción de las solicitudes que no esperaron para ser procesadas.
 
 - La opción 1 es más costosa que la segunda opción y la empresa sólo acepta realizar la inversión si el tiempo medio
que demora en resolver cada solicitud (tiempo en fila + tiempo de procesamiento) es como mínimo 50% menor
que la opción 2. ¿Qué solución le recomienda?

Comenzamos realizando la simulación para 100000 solicitudes procesadas en el escenario en que el tenemos una base de datos distribuida con dos nodos. Se generarán 100000 muestras correspondientes a los tiempos de arribo de las solicitudes según la tasa indicada en el enunciado, que luego serán divididas entre aquellas que son procesadas por la primera base y la segunda.

In [79]:
def simular_procesamiento_base_de_datos(tiempos_de_arribo, media):
    z = np.random.exponential(media, len(tiempos_de_arribo))
    t_actual = 0
    tiempos_de_espera = []
    tiempos_totales_de_procesamiento = []
    for index, t_arribo in enumerate(tiempos_de_arribo):
        if t_arribo < t_actual:
            tiempos_de_espera.append(t_actual - t_arribo)
            t_actual += z[index]
            tiempos_totales_de_procesamiento.append(t_actual - t_arribo + z[index])
        else:
            tiempos_de_espera.append(0)
            t_actual = t_arribo + z[index]
            tiempos_totales_de_procesamiento.append(z[index])
    return sum(tiempos_de_espera) / len(tiempos_de_espera), len([t for t in tiempos_de_espera if t == 0]) / len(tiempos_de_arribo), sum(tiempos_totales_de_procesamiento) / len(tiempos_totales_de_procesamiento)

In [83]:
np.random.seed(32)
media_arribos = 4            
tasa_de_arribo = 1/media_arribos
N = 100000
p = 0.6
media_base_1 = 0.7
media_base_2 = 1

#Generamos N tiempos de arribo.
z = np.random.exponential(1/tasa_de_arribo, N)

#Calculamos los tiempos de arribo.
t = np.concatenate(([0], np.cumsum(z)), axis=None)

#Separamos el proceso segun la base de datos donde sea procesada. Generamos probabilidades aleatorias y las iteramos.
u = np.random.rand(N)

t_base_1 = [0]
t_base_2 = [0]

for i in range(0, N):
    if u[i] < p:
        t_base_1.append(t[i])
    else:
        t_base_2.append(t[i])

tiempo_medio_de_espera_base_1, fraccion_sin_espera_base_1, tiempo_medio_procesamiento_base_1 = simular_procesamiento_base_de_datos(t_base_1, media_base_1)
tiempo_medio_de_espera_base_2, fraccion_sin_espera_base_2, tiempo_medio_procesamiento_base_2 = simular_procesamiento_base_de_datos(t_base_2, media_base_2)

Utilizando un promedio ponderado, calculamos cual es el tiempo medio de entrega y la fracción de solicitudes que no debieron esperar para ser procesadas:

In [84]:
tiempo_promedio_de_espera = (tiempo_medio_de_espera_base_1 * len(t_base_1) + tiempo_medio_de_espera_base_2 * len(t_base_2)) / N
fraccion_sin_espera = fraccion_sin_espera_base_1 * len(t_base_1) / N + fraccion_sin_espera_base_2 * len(t_base_2) / N
tiempo_promedio_de_procesamiento = (tiempo_medio_procesamiento_base_1 * len(t_base_1) + tiempo_medio_procesamiento_base_2 * len(t_base_2)) / N

print("El tiempo medio de espera para una base distribuida con dos nodos es {} segundos.".format(tiempo_promedio_de_espera))
print("La fracción de solicitudes que no deben esperar en caso de que haya una base distribuida con dos nodos es {}.".format(fraccion_sin_espera))
print("El tiempo medio de procesamiento para una solicitud utilizando una base distribuida con dos nodos es {} segundos.".format(tiempo_promedio_de_procesamiento))

El tiempo medio de espera para una base distribuida con dos nodos es 0.09396557408749351 segundos.
La fracción de solicitudes que no deben esperar en caso de que haya una base distribuida con dos nodos es 0.89839.
El tiempo medio de procesamiento para una solicitud utilizando una base distribuida con dos nodos es 1.0006867507267798 segundos.


Ahora, obtengamos las mismas mediciones pero realizando la simulación para el segundo escenario, en donde tenemos una única base de datos centralizada.

In [86]:
np.random.seed(32)
media_arribos = 4            
tasa_de_arribo = 1/media_arribos
N = 100000
media_base = 0.8

#Generamos N tiempos de arribo.
z = np.random.exponential(1/tasa_de_arribo, N)

#Calculamos los tiempos de arribo.
t = np.concatenate(([0], np.cumsum(z)), axis=None)

tiempo_medio_de_espera_base, fraccion_sin_espera_base, tiempo_promedio_de_procesamiento_base = simular_procesamiento_base_de_datos(t, media_base)
print("El tiempo medio de espera para una base centralizada es {} segundos.".format(tiempo_medio_de_espera_base))
print("La fracción de solicitudes que no deben esperar en caso de que haya una base centralizada es {}.".format(fraccion_sin_espera_base))
print("El tiempo medio de procesamiento para una solicitud utilizando una base centralizada es {} segundos.".format(tiempo_promedio_de_procesamiento_base))

El tiempo medio de espera para una base centralizada es 0.2007744701672654 segundos.
La fracción de solicitudes que no deben esperar en caso de que haya una base centralizada es 0.7984520154798452.
El tiempo medio de procesamiento para una solicitud utilizando una base centralizada es 1.1658036051222367 segundos.


Vemos que, en el caso de utilizar una base distribuida con dos nodos a diferencia de una centralizada, hay una disminución del tiempo de procesamiento de una solicitud promedio, pasando de 1.1658036051222367 segundos a 1.0006867507267798 segundos. Sin embargo, esto no es suficiente para cumplir con lo solicitado, una disminución del tiempo del 50% (el tiempo de respuesta promedio de la base distribuida debería estar debajo de 0.5829018025611183 segundos).

Por lo tanto, le recomendaríamos a la empresa que no realice la inversión y mantenga su infraesctructura actual, ya que al realizar al utilizar una base distribuida no se puede cumplir con el tiempo de procesamiento deseado.